In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import telebot

# Параметры изображения
image_height = 224
image_width = 224

# Путь к вашим весам модели
model_path = "/content/drive/MyDrive/Трубчик МО/sea_animals_classification_model.h5"

# Токен вашего бота
bot_token = '7056109654:AAEB-YncKZSqf35L1Nld8-baIisfKkaVmFI'
bot = telebot.TeleBot(bot_token)

# Загрузка модели
resnet_model = tf.keras.models.load_model(model_path)

# Ответ на команду /start
@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.send_message(message.chat.id, "Привет! Я бот для классификации рыб. Пожалуйста, отправьте мне изображение для анализа в формате .jpeg.")

# Обработка изображений
@bot.message_handler(content_types=['photo'])
def handle_image(message):
    # Сохраняем изображение
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open("downloaded_image.jpeg", "wb") as new_file:
        new_file.write(downloaded_file)

    # Загружаем изображение и делаем предобработку
    img_path = "downloaded_image.jpeg"
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # добавляем размерность пакета
    img_array /= 255.  # предобработка

    # Получаем предсказания модели
    predictions = resnet_model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    confidence = predictions[0][predicted_class_index]

    # Отправляем ответ пользователю
    if confidence >= 0.8:
        class_names = ['Clams', 'Crabs', 'Dolphin']  # Названия классов
        predicted_class_name = class_names[predicted_class_index]
        response = f"Предсказанный класс: {predicted_class_name}\nУверенность: {confidence*100:.2f}%"
        bot.send_message(message.chat.id, response)
    else:
        bot.send_message(message.chat.id, "Я не могу определить класс рыбы на изображении.")

# Запуск бота
if __name__ == "__main__":
    bot.polling(none_stop=True)
